Stokes Solver
=======

We want to solve the following Stokes block system.
\\[
\begin{bmatrix}
 K & G \\
 G^T & C
\end{bmatrix}
\begin{bmatrix}
  u\\
  p
\end{bmatrix}
=
\begin{bmatrix}
 f\\ 
 h
\end{bmatrix}.
\\]

If we apply Gaussian elimination to the above as a 2x2 block matrix system
we can write this as:

\\[
\begin{bmatrix}
  K & G\\
  0 & S
\end{bmatrix}
\begin{bmatrix}
  u\\
  p
\end{bmatrix}
=
\begin{bmatrix}
 f\\ 
 \hat{h}
\end{bmatrix},
\\]

where $S=G^{T}K^{-1}G-C$ is the Schur complement and $\hat{h}=G^{T}K^{-1}f -h$.

This system is now solved first for the pressure using the Schur complement matrix, $S$. Then a backsolve for the
velocity gives the complete solution.

Note that wherever $K^{-1}$ appears, the inverse is never explicitly calculated but is achieved via
a [PETSc](http://www.mcs.anl.gov/petsc/) solve method. While solving for the pressure, there are necessarily solves using $K$ inside of the matrix $S$.
We often refer to these as 'inner' solves.

Basic usage of the Stokes solver class involves being able to easily set up the inner solves in a few different ways
(Setting up the pressure solve is more advanced).

To illustrate some basic usage let's set up a simple problem to solve.

In [1]:
import underworld as uw
import math
from underworld import function as fn

res=128
mesh = uw.mesh.FeMesh_Cartesian("Q1/DQ0", (res,res), (0.,0.), (1.,1.))
velocityField = uw.mesh.MeshVariable(mesh,2)
velocityField.data[:] = (0.,0.)

pressureField = uw.mesh.MeshVariable(mesh.subMesh,1)
pressureField.data[:] = 0.

# Freeslip bc's
IWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
JWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
freeslip = uw.conditions.DirichletCondition(velocityField, (IWalls, JWalls))
# We are going to make use of one of the existing analytic solutions so that we may easily
# obtain functions for a viscosity profile and forcing terms.
# Exact solution solCx with defaults
sol = fn.analytic.SolCx()
stokesSystem = uw.systems.Stokes(velocityField,pressureField,sol.fn_viscosity,sol.fn_bodyforce,conditions=[freeslip,])


Now we create a solver.
The Solver class will automatically return a Stokes solver given a Stokes system.

In [2]:
solver=uw.systems.Solver(stokesSystem)

The Stokes solver will use multigrid as a preconditioner along with PETSc's
'fgmres' Krylov method by default for the 'inner' solve.

Let's run the solver now.

In [3]:
solver.solve()

Now let's set up the inner solve to do a direct solve (this will not work in parallel).

In [4]:
solver.set_inner_method("lu")

In [5]:
solver.solve()

Let's run underworld's help function on the solver.configure function.

In [6]:
uw.help(solver.set_inner_method)

Object docstring:


        Configure velocity/inner solver (A11 PETSc prefix).

        solve_type can be one of:

        mg          : Geometric multigrid (default).
        mumps       : MUMPS parallel direct solver.
        superludist : SuperLU parallel direct solver.
        superlu     : SuperLU direct solver (serial only).
        lu          : LU direct solver (serial only).
        
Object initialiser docstring:

x.__init__(...) initializes x; see help(type(x)) for signature


If PETSc has been compiled with mumps we can also set up the inner solve this way.
Mumps is a direct solve that will work in parallel.

In [7]:
solver.set_inner_method("mumps")

In [8]:
solver.solve()

Now let's go back to using multigrid.

In [9]:
solver.set_inner_method("mg")
solver.solve()

Let's run help on the solver.

In [10]:
uw.help(solver)

Object docstring:


    The Block Stokes Schur Complement Solver:
    This solves the saddle-point system
    
    [ K  G][u] = [f]
    [ G' C][p]   [h]

    via a Schur complement method.
    
    We first solve:
      a: S*p= G'*Ki*f - h,

    where S = G'*Ki*G-C and Ki = inverse of K.

    Then we backsolve for the velocity
      b: K*u = f - G*p.
    
    The effect of the inverse of K in (a) is obtained via a KSPSolve in PETSc.
    This has the prefix 'A11' (often called the 'inner' solve)
    
    The solve in (a) for the pressure has prefix 'scr'.
    
    Assuming the returned solver is called 'solver':

    It is possible to configure these solves individually via the
      solver.options.A11
    and
      solver.options.scr
    dictionaries. 
    
    Try uw.help(solver.options.A11) for some details.
    
    Common configurations are provided via the
    solver.set_inner_method() function.
    
    solver.set_inner_method("mg") sets up a multigrid solve for the inner solve. 